In this notebook we are primarly used RAG(Retrieval Augmented Generation) to overcome traditional LLM limitations.The RAG primarly consist of three aspects:
Indexing
Retrieval
Generation
Indexing happens ahead of time, and allows you to quickly look up relevant information at query-time. When a query comes in, you retrieve relevant documents, combine them with your instructions and the user's query, and have the LLM generate a tailored answer in natural language using the supplied information. All these will help us to retrieve reason behind earth quakes.
**Documents are the items that are in the database. They are inserted first, and later retrieved. Queries are the textual search terms and can be simple keywords or textual descriptions of the desired documents**

We will use the Gemini API to create a vector database, retrieve answers to questions from the database and generate a final answer.
**Setup**
First, installing ChromaDB and the Gemini API Python SDK with required libraries.

In [ ]:
%pip install -U -q "google-genai>=1.0.0"
%pip install -q chromadb

In [ ]:
import textwrap
import chromadb
import numpy as np
import pandas as pd

from IPython.display import Markdown
from chromadb import Documents, EmbeddingFunction, Embeddings

To make the key available through colab(Left hand side SIGN) -secrets, & add  key or enable it for this notebook.

In [ ]:
from kaggle_secrets import UserSecretsClient

GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")

client = genai.Client(api_key=GOOGLE_API_KEY)

response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain Earthquakes to me like I'm a kid.")

print(response.text)

**Exploring available models**
We are using the embedContent API method to calculate embeddings in this notebook. We will check existing embeded models for processing text.

In [ ]:
for m in client.models.list():
  if 'embedContent' in m.supported_actions:
    print(m.name)

We are creating small set of documents below and there by creates an embedding database.The below 4 documents will help us to understand the resons behinfd earthquakes and specific reason of Mynmar & Thailand earthquake.

In [ ]:
DOCUMENT1 = """
Myanmar is considered to be one of the most geologically "active" areas in the world because it sits on top of the convergence of four of these tectonic plates - the Eurasian plate, the Indian plate, the Sunda plate and the Burma microplate.
"""
DOCUMENT2 = """
There is a major fault called the Sagaing fault, which cuts right through Myanmar north to south and is more than 1,200km (746 miles) long.As the plates move past each other, they can become stuck, building friction until it is suddenly released and the earth shifts, causing an earthquake.
"""
DOCUMENT3 = """
Early data suggests that the movement that caused Friday's 7.7-magnitude earthquake was a "strike-slip" - where two blocks move horizontally along each other.Because the fracture "unzipped" towards the south, it also directed this piled up energy towards the Thai capital, Bangkok, and this is why the earthquake had such an impact so far away.
"""
DOCUMENT4 = """
plates move past each other horizontally at different speeds. While this causes “strike slip” quakes that are normally less powerful than those seen in “subduction zones” like Sumatra, where one plate slides under another, they can still reach magnitudes of 7 to 8
"""
documents = [DOCUMENT1, DOCUMENT2, DOCUMENT3,DOCUMENT4]


We are Creating a custom function to generate embeddings with the Gemini API. In this task, we are implementing a **retrieval system**, so the **task_type** for generating the document embeddings is retrieval_document. Later, you will use **retrieval_query** for the query embeddings.

In [ ]:
from google.genai import types

class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    EMBEDDING_MODEL_ID = "models/embedding-001"  # @param ["models/embedding-001", "models/text-embedding-004", "models/gemini-embedding-exp-03-07", "models/gemini-embedding-exp"] {"allow-input": true, "isTemplate": true}
    title = "Custom query"
    response = client.models.embed_content(
        model=EMBEDDING_MODEL_ID,
        contents=input,
        config=types.EmbedContentConfig(
          task_type="retrieval_document",
          title=title
        )
    )

    return response.embeddings[0].values

Now we are creating a Chroma database client which uses the **GeminiEmbeddingFunction** and then populating the database with the documents we defined above.

In [ ]:
def create_chroma_db(documents, name):
  chroma_client = chromadb.Client()
  db = chroma_client.create_collection(
      name=name,
      embedding_function=GeminiEmbeddingFunction()
  )

  for i, d in enumerate(documents):
    db.add(
      documents=d,
      ids=str(i)
    )
  return db

Here we are creating vector db for retrieval

In [ ]:
db = create_chroma_db(documents, "Recent_Earthquakes_Mynmar_Thailand")

Here we are retrieving the stored data — then formatting it into a nice pandas DataFrame for easy inspection.

In [ ]:
sample_data = db.get(include=['documents', 'embeddings'])

df = pd.DataFrame({
    "IDs": sample_data['ids'][:4],
    "Documents": sample_data['documents'][:4],
    "Embeddings": [str(emb)[:100] + "..." for emb in sample_data['embeddings'][:4]]  # Truncate embeddings
})

print(df)

The belwo function is designed to retrieve the most relevant passage from your stored documents based on a text query using semantic similarity.

In [ ]:
def get_relevant_passage(query, db):
  passage = db.query(query_texts=[query], n_results=1)['documents'][0][0]
  return passage

The below searches the vector database (db) for the document most semantically similar to the phrase "major fault".

The function returns the top matching text passage, which is saved into the variable passage.

In [ ]:
# Perform embedding search
passage = get_relevant_passage("major fault", db)
Markdown(passage)

**Augmented generation**: Answer the question¶
Now that you have found a relevant passage from the set of documents (the retrieval step), you can now assemble a generation prompt to have the Gemini API generate a final answer. Note that in this example only a single passage was retrieved.Here we are adding useful prompts for retrieval purpose  

In [ ]:
def make_prompt(query, relevant_passage):
  escaped = relevant_passage.replace("'", "").replace('"', "").replace("\n", " ")
  prompt = ("""
    You are a helpful and informative bot that answers questions using
    text from the reference passage included below.
    Be sure to respond in a complete sentence, being comprehensive,
    including all relevant background information.
    However, you are talking to a non-technical audience, so be sure to
    break down complicated concepts and strike a friendly
    and converstional tone. If the passage is irrelevant to the answer,
    you may ignore it.
    QUESTION: '{query}'
    PASSAGE: '{relevant_passage}'

    ANSWER:
  """).format(query=query, relevant_passage=escaped)

  return prompt

In [ ]:
query = "why plates moving?"
prompt = make_prompt(query, passage)
Markdown(prompt)

Now we are using **generate_content** method to to get an answer to the question.

In [ ]:
MODEL_ID = "gemini-2.0-flash"  # @param ["gemini-2.0-flash-lite", "gemini-2.0-flash", "gemini-2.5-pro-exp-03-25"] {"allow-input": true, "isTemplate": true}
answer = client.models.generate_content(
    model = MODEL_ID,
    contents = prompt
)
Markdown(answer.text)

The above excercise will help us to Q&A nd relevance of RAG for retreival generation.